## Case Study #1 - Danny's Diner


#### Problem Statement
Danny wants to use the data to answer a few simple questions about his customers, especially about their visiting patterns, how much money they’ve spent and also which menu items are their favourite. Having this deeper connection with his customers will help him deliver a better and more personalised experience for his loyal customers.

He plans on using these insights to help him decide whether he should expand the existing customer loyalty program - additionally he needs help to generate some basic datasets so his team can easily inspect the data without needing to use SQL.

Danny has provided you with a sample of his overall customer data due to privacy issues - but he hopes that these examples are enough for you to write fully functioning SQL queries to help him answer his questions!

Danny has shared with you 3 key datasets for this case study: `sales`, `menu`, and `members`

#### Entity Relationship Diagram

![week1.png](week1.png)

Import modules

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as sql
from sqlalchemy import text
pd.set_option('display.max_columns', None)

Initialize SQL

In [2]:
conn = sql.connect("week1.db")
cursor = conn.cursor() 
if os.stat("week1.db").st_size == 0:
    with open('week1-sql.txt','r') as file:
        script = file.read()
        script = script.replace('\n', ' ')
    cursor.executescript(script)

Verify tables

In [3]:
query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor.execute(query)
tables = [table[0] for table in cursor.fetchall()]
tables
print(f'The tables in the database are: {", ".join(tables)}')

The tables in the database are: sales, menu, members


Fetch table information

In [4]:
for table in tables:
    print("=================================")
    print(f'Table [{table}]')
    df = pd.read_sql_query(f'SELECT * FROM {table}', conn)
    print(f'Dimensions: {df.shape[0]} rows x {df.shape[1]} columns\n')
    print(df.head())
    info_df = pd.DataFrame.from_dict({'Datatypes':df.dtypes, 'NULL count':df.isna().sum()})
    print()
    print(info_df)
    print()

Table [sales]
Dimensions: 15 rows x 3 columns

  customer_id  order_date  product_id
0           A  2021-01-01           1
1           A  2021-01-01           2
2           A  2021-01-07           2
3           A  2021-01-10           3
4           A  2021-01-11           3

            Datatypes  NULL count
customer_id    object           0
order_date     object           0
product_id      int64           0

Table [menu]
Dimensions: 3 rows x 3 columns

   product_id product_name  price
0           1        sushi     10
1           2        curry     15
2           3        ramen     12

             Datatypes  NULL count
product_id       int64           0
product_name    object           0
price            int64           0

Table [members]
Dimensions: 2 rows x 2 columns

  customer_id   join_date
0           A  2021-01-07
1           B  2021-01-09

            Datatypes  NULL count
customer_id    object           0
join_date      object           0



## Case Study Questions

Q1: What is the total amount each customer spent at the restaurant?

In [5]:
# Sample
query = """          
    SELECT 
        sales.customer_id, 
        SUM(menu.price) AS total_sales
    FROM sales
    INNER JOIN menu
        ON sales.product_id = menu.product_id
    GROUP BY sales.customer_id
    ORDER BY sales.customer_id ASC;           
"""
result = pd.read_sql_query(query, conn)
result

,customer_id,total_sales
0,A,76
1,B,74
2,C,36


Q2: How many days has each customer visited the restaurant?

In [6]:
# Sample
query = """          
    SELECT 
        customer_id, 
        COUNT(DISTINCT order_date) AS num_visits
    FROM sales
    GROUP BY customer_id;
"""
result = pd.read_sql_query(query, conn)
result
# Note: DISTINCT used as there can be multiple orders per day

,customer_id,num_visits
0,A,4
1,B,6
2,C,2


Q3: What was the first item from the menu purchased by each customer?

In [7]:
# Sample
query = """
    SELECT 
        customer_id, 
        product_name
    FROM (
        SELECT 
            sales.customer_id, 
            sales.order_date, 
            menu.product_name,
            DENSE_RANK() OVER (
            PARTITION BY sales.customer_id 
            ORDER BY sales.order_date) AS rank
        FROM  sales
        INNER JOIN  menu
            ON sales.product_id = menu.product_id
    ) AS ordered_sales
    WHERE rank = 1
    GROUP BY customer_id, product_name;   
"""
result = pd.read_sql_query(query, conn)
result

,customer_id,product_name
0,A,curry
1,A,sushi
2,B,curry
3,C,ramen


Q4: What is the most purchased item on the menu and how many times was it purchased by all customers?

In [8]:
# Sample
query = """          
    SELECT 
        menu.product_name,
        COUNT(sales.product_id) AS most_purchased_item
    FROM  sales
    INNER JOIN  menu
        ON sales.product_id = menu.product_id
    GROUP BY menu.product_name
    ORDER BY most_purchased_item DESC
    LIMIT 1;       
"""
result = pd.read_sql_query(query, conn)
result

,product_name,most_purchased_item
0,ramen,8


Q5: Which item was the most popular for each customer?

In [9]:
# Sample
query = """          
    WITH most_popular AS (
        SELECT 
            sales.customer_id, 
            menu.product_name, 
            COUNT(menu.product_id) AS order_count,
            DENSE_RANK() OVER (
            PARTITION BY sales.customer_id 
            ORDER BY COUNT(sales.customer_id) DESC) AS rank
        FROM  menu
        INNER JOIN  sales
            ON menu.product_id = sales.product_id
        GROUP BY sales.customer_id, menu.product_name
    )

    SELECT 
        customer_id, 
        product_name, 
        order_count
    FROM most_popular 
    WHERE rank = 1;       
"""
result = pd.read_sql_query(query, conn)
result

,customer_id,product_name,order_count
0,A,ramen,3
1,B,sushi,2
2,B,ramen,2
3,B,curry,2
4,C,ramen,3


Q6: Which item was purchased first by the customer after they became a member?

In [10]:
# Sample
query = """          
    WITH joined_as_member AS (
    SELECT
        members.customer_id, 
        sales.product_id,
        ROW_NUMBER() OVER (
        PARTITION BY members.customer_id
        ORDER BY sales.order_date) AS row_num
    FROM  members
    INNER JOIN  sales
        ON members.customer_id = sales.customer_id
        AND sales.order_date > members.join_date
    )

    SELECT 
        customer_id, 
        product_name 
    FROM joined_as_member
    INNER JOIN  menu
        ON joined_as_member.product_id = menu.product_id
    WHERE row_num = 1
    ORDER BY customer_id ASC; 
"""
result = pd.read_sql_query(query, conn)
result

,customer_id,product_name
0,A,ramen
1,B,sushi


Q7: Which item was purchased just before the customer became a member?

In [11]:
# Sample
query = """          
    WITH purchased_prior_member AS (
    SELECT 
        members.customer_id, 
        sales.product_id,
        ROW_NUMBER() OVER (
        PARTITION BY members.customer_id
        ORDER BY sales.order_date DESC) AS rank
    FROM  members
    INNER JOIN  sales
        ON members.customer_id = sales.customer_id
        AND sales.order_date < members.join_date
    )

    SELECT 
        p_member.customer_id, 
        menu.product_name 
    FROM purchased_prior_member AS p_member
    INNER JOIN  menu
        ON p_member.product_id = menu.product_id
    WHERE rank = 1
    ORDER BY p_member.customer_id ASC; 
"""
result = pd.read_sql_query(query, conn)
result

,customer_id,product_name
0,A,sushi
1,B,sushi


Q8: What is the total items and amount spent for each member before they became a member?

In [12]:
# Sample
query = """          
    SELECT 
        sales.customer_id, 
        COUNT(sales.product_id) AS total_items, 
        SUM(menu.price) AS total_sales
    FROM  sales
    INNER JOIN  members
        ON sales.customer_id = members.customer_id
        AND sales.order_date < members.join_date
    INNER JOIN  menu
        ON sales.product_id = menu.product_id
    GROUP BY sales.customer_id
    ORDER BY sales.customer_id;      
"""
result = pd.read_sql_query(query, conn)
result

,customer_id,total_items,total_sales
0,A,2,25
1,B,3,40


Q9: If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [13]:
# Sample
query = """          
    WITH points_cte AS (
    SELECT 
        menu.product_id, 
        CASE
        WHEN product_id = 1 THEN price * 20
        ELSE price * 10 END AS points
    FROM  menu
    )

    SELECT 
        sales.customer_id, 
        SUM(points_cte.points) AS total_points
    FROM  sales
    INNER JOIN points_cte
        ON sales.product_id = points_cte.product_id
    GROUP BY sales.customer_id
    ORDER BY sales.customer_id;    
"""
result = pd.read_sql_query(query, conn)
result

,customer_id,total_points
0,A,860
1,B,940
2,C,360


Q10: In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?


In [14]:
# Sample
query = """          
    WITH dates_cte AS (
    SELECT 
        customer_id, 
        join_date, 
        DATETIME(join_date, '+6 days') AS valid_date, 
        DATETIME('2021-01-31') AS last_date
    FROM  members
    )

    SELECT 
        sales.customer_id, 
        SUM(CASE
            WHEN menu.product_name = 'sushi' THEN 2 * 10 * menu.price
            WHEN sales.order_date BETWEEN dates.join_date AND dates.valid_date THEN 2 * 10 * menu.price
            ELSE 10 * menu.price END) AS points
    FROM  sales
    INNER JOIN dates_cte AS dates
        ON sales.customer_id = dates.customer_id
        AND dates.join_date <= sales.order_date
        AND sales.order_date <= dates.last_date
    INNER JOIN  menu
        ON sales.product_id = menu.product_id
    GROUP BY sales.customer_id; 
"""
result = pd.read_sql_query(query, conn)
result

,customer_id,points
0,A,1020
1,B,320


Q11: Recreate the following table output with the following columns: customer_id, order_date, product_name, price, member (N if customer made the order before he joined as a member, and Y otherwise) 

In [15]:
# Sample
query = """          
    SELECT 
        sales.customer_id, 
        sales.order_date,  
        menu.product_name, 
        menu.price,
        CASE
            WHEN members.join_date > sales.order_date THEN 'N'
            WHEN members.join_date <= sales.order_date THEN 'Y'
            ELSE 'N' END AS member_status
    FROM  sales
    LEFT JOIN  members
        ON sales.customer_id = members.customer_id
    INNER JOIN  menu
        ON sales.product_id = menu.product_id
    ORDER BY members.customer_id, sales.order_date
"""
result = pd.read_sql_query(query, conn)
result

,customer_id,order_date,product_name,price,member_status
0,C,2021-01-01,ramen,12,N
1,C,2021-01-01,ramen,12,N
2,C,2021-01-07,ramen,12,N
3,A,2021-01-01,sushi,10,N
4,A,2021-01-01,curry,15,N
5,A,2021-01-07,curry,15,Y
6,A,2021-01-10,ramen,12,Y
7,A,2021-01-11,ramen,12,Y
8,A,2021-01-11,ramen,12,Y
9,B,2021-01-01,curry,15,N


Q12: From the table in Q11, add a ranking column where it is NULL for all non member rows, and 1, 2, ... representing order number ordered by order date of the customer.

In [16]:
# Sample
query = """          
    WITH customers_data AS (
        SELECT 
            sales.customer_id, 
            sales.order_date,  
            menu.product_name, 
            menu.price,
            CASE
                WHEN members.join_date > sales.order_date THEN 'N'
                WHEN members.join_date <= sales.order_date THEN 'Y'
            ELSE 'N' END AS member_status
        FROM  sales
        LEFT JOIN  members
            ON sales.customer_id = members.customer_id
        INNER JOIN  menu
            ON sales.product_id = menu.product_id
    )

    SELECT 
        *, 
        CASE
            WHEN member_status = 'N' then NULL
            ELSE RANK () OVER (
            PARTITION BY customer_id, member_status
            ORDER BY order_date
        ) END AS ranking
    FROM customers_data;
"""
result = pd.read_sql_query(query, conn)
result

,customer_id,order_date,product_name,price,member_status,ranking
0,A,2021-01-01,sushi,10,N,NaN
1,A,2021-01-01,curry,15,N,NaN
2,A,2021-01-07,curry,15,Y,1.0
3,A,2021-01-10,ramen,12,Y,2.0
4,A,2021-01-11,ramen,12,Y,3.0
5,A,2021-01-11,ramen,12,Y,3.0
6,B,2021-01-01,curry,15,N,NaN
7,B,2021-01-02,curry,15,N,NaN
8,B,2021-01-04,sushi,10,N,NaN
9,B,2021-01-11,sushi,10,Y,1.0
